# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook-tagged.dat` from Blackboard

You may process the file using the following code. You should inspect the code to ensure that you understand its operation.

_Note:  [the "yield" statement](http://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do-in-python#231855)  makes a function into an "iterator"_

_Note: There are snippets of code to help you build your answers. These are provided as a guide and do not need to be followed exactly.  
Comments like _"# Write your code here."_ indicate where you could write your code._

In [1]:
from nltk.tokenize.regexp import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

class SpellingSentence:
    """Store a sentence in original form and any spelling errors."""
    def __init__(self, text, spans, error_spans, corrections):
        """Constructor parameters are as follows:
        text: the original sentence.
        spans: an iterable that returns a span (start and end indices) for each word in the text.
        error_spans: a list of spans for mis-spelled words.
        corrections: a list of corresponding correctly spelled words.
        """
        self.text = text
        self.spans = list(spans)
        self.error_spans = error_spans
        self.corrections = corrections
        
    def __str__(self):
        return self.text
        
    def tokens_with_errors(self):
        """Returns an iterator over the original (possibly mis-spelled) words as strings."""
        for x, y in self.spans:
            yield self.text[x:y]
 
    def tokens_corrected(self):
        """Returns an iterator over the corrected words as strings."""
        token_spans = []
        for s1 in self.spans:
            is_not_in_error_span = True
            for s2 in self.error_spans:
                is_not_in_error_span = (is_not_in_error_span and
                                        not (s1[0] >= s2[0] and s1[1] <= s2[1]))
            if is_not_in_error_span:
                token_spans.append(s1)
                    
        token_spans = token_spans + self.error_spans
        token_spans = sorted(token_spans, key = lambda s1: s1[0])
        
        for x,y in token_spans:
            if (x, y) in self.error_spans:
                yield self.corrections[self.error_spans.index((x, y))]
            else:
                yield self.text[x:y]
    
    def errors(self):
        """Returns an iterator over the mis-spelled words as strings."""
        for x, y in self.error_spans:
            yield self.text[x:y]
    

def read_sentence(sentence):
    """Reads a sentence from the holbrook spelling corpus (as text), 
    returning a SpellingSentence instance for that senetence."""
    error_spans = []
    corrections = []
    while "<ERR" in sentence:
        i = sentence.index("<ERR")
        j = sentence.index(">")
        correction = sentence[(i+10):j]
        k = sentence.index("</ERR>")
        if i > 0:
            sentence = sentence[:(i-1)] + sentence[(j+1):(k-1)] + sentence[(k+6):]
        else:
            sentence = sentence[(j+1):(k-1)] + sentence[(k+6):]
        error_spans.append((i, k - (j - i  + 3)))
        corrections.append(correction)
    return SpellingSentence(sentence, tokenizer.span_tokenize(sentence), 
                            error_spans, corrections)
       
    

        
# Example usage:
ss = read_sentence("My <ERR targ=sister> siter </ERR> <ERR targ=goes> go </ERR> to Tonbury.")

print(list(ss.tokens_with_errors()))
print(list(ss.tokens_corrected()))
print(ss.error_spans)
print(ss.spans)
print(list(ss.errors()))
print(ss.corrections)
        
        

['My', 'siter', 'go', 'to', 'Tonbury', '.']
['My', 'sister', 'goes', 'to', 'Tonbury', '.']
[(3, 8), (9, 11)]
[(0, 2), (3, 8), (9, 11), (12, 14), (15, 22), (22, 23)]
['siter', 'go']
['sister', 'goes']


You can load all the data as follows:

_Note: you need to have downloaded the file _"holbrook-tagged.dat"_ from blackboard into your notebook directory._

In [2]:


data = [read_sentence(line) for line in open("holbrook-tagged.dat").readlines()]

test = data[1:100]
train = data[101:]

#train = [d.text for d in train]
#print(train_set)
test_set = [d.text for d in test]
#print(test_set)
#test     

\## **Task 1**: 
Calculate the frequency, *ignoring case*, of all words and bigrams (sequences of two words) satisfying the following assertions from the corrected *training* sentences:

In [3]:
import numpy as np
from nltk import bigrams

def unigram(word):
    # ** Don't go through the whole corpus every time! Better to pre-compute values and look up here (-1)
    z=0
    for each_line in train:
        z+= str(each_line).lower().split(" ").count(word)
    return z

from collections import Counter
from nltk.collocations import *

def bigram(words):
    d={}
    s={}
    z=0
    words_bi = tuple(words.split(" "))
    #words_bi = words
    #print(words_bi)
    # ** You generate all bigrams to calculate one frequency. Better to have precomputed (-1)
    for each_line in train:
        d= list(bigrams(str(each_line).lower().split(" ")))
        s=Counter(d)
        z=z+s[words_bi] 
                                  
    return z

uni=unigram("the") 
print("Frequency of the unigram word 'the' :", uni)
bi =bigram("it will")
print("Frequency of the bigram words 'it will' :",bi)

Frequency of the unigram word 'the' : 1529
Frequency of the bigram words 'it will' : 9


## **Task 2**: 
Using the following as a basis find all words that have the minimal edit distance from a given word (i.e., there does not exist another word with lower edit distance in the corrected training corpus):

*Note: [Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a way of quantifying how dissimilar two strings are to one another by counting the minimum number of operations required to transform one string into the other*

In [4]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

all_words = set()

for line in train:
    all_words.update(line.tokens_corrected())
    


4


In [5]:
import numpy as np

def get_candidates(token):
    ed = []
    d1 = []

    for t in all_words:
        ed.append(edit_distance(token, t))
    min_ed = min(ed)
    
    # ** you should make this conversion earlier and use all_l instead of all_words in the above loop also (-1)
    # ** this is because you cannot rely on the iteration order of sets (like all_words)
    all_1=list(all_words)
    # ** why are you converting first to a list, then an array? Use one or the other, probably a list here
    myarray = np.asarray(all_1)
    for i in range(len(ed)):
        if ed[i] == min_ed:
            d1.append(myarray[i])
        
    return d1


        
# get_candidates() should find the following
#assert get_candidates("calld") = ['calls', 'call', 'called']
print("Words with minimal edit distance for the word 'calld' are : ",get_candidates("calld"))

Words with minimal edit distance for the word 'calld' are :  ['called', 'call', 'calls']


## **Task 3**: 
Write a function that corrects spelling based on n-gram language model probabilities. It should work as follows

* Check if each word is the same as a correctly spelled word from the corpus
* If not, find the corpus words with minimum edit distance to that word
* If multiple corpus words have the same edit distance
    * Choose the most likely word based on the bigram probability, recall:
        * $p(w_n|w_{n-1}) = \frac{c(w_{n-1}w_n)}{c(w_{n-1}\cdot)}$ where $c()$ is the count in the corpus
    * If the bigram probability does not exist (or the spelling error is in the first word), use the unigram probability, recall:
        * $p(w_n) = \frac{c(w_n)}{N}$ where $N$ is the total number of *tokens* (not types) in the corpus

In [331]:
# ** you need to reset token_incorrect INSIDE the function, else it just keeps getting bigger with each call! (-1)
token_incorrect = []
def correct(tokens):
    for token in tokens:
        j=1
        for i in all_words:
            if token == i:
                j=0
        if j==1:
            token_incorrect.append(token)
    #print("Incorrect token is : ", token_incorrect)             
    return token_incorrect
    # ** Why do you end the function here! (-1)

is_misspelled=correct(["so", "they", "calld", "it", "the", "murder", "car"])

test_ex = ["so", "they", "calld", "it", "the", "murder", "car"]
#test_ex = ['NIGEL', 'THRUSH', 'page', '48']
print("Words mispelled : ", is_misspelled)

# if is_misspelled(token):
# ** youre converting a list is_misspelled to a string - you should get_candidates() for each element of it! (-2)
# ** all the code below should be in a loop over words in is_misspelled...
candidates = get_candidates(str(is_misspelled))
print("Candidates with minimum edit distance :",candidates)
error_index=test_ex.index(is_misspelled[0])

from operator import itemgetter
if len(candidates) > 1:
    # ** is_misspelled is a list, text_ex[0] a string ... this if is always true!! (-1)
    if is_misspelled != test_ex[0]:
        
         # Find candidate with highest bigram value
        probability_bigram = {}
        for i in range(len(candidates)):
            bigram_words= test_ex[error_index-1] + " " + candidates[i]
            #print(bigram_words)
            probability_bigram[i] = bigram(bigram_words)/unigram(test_ex[error_index-1])
            
        # ** an unusual way to get there, but it works! 
        p=Counter(probability_bigram).most_common()[0]
        
        print("Corrected word with high bigram probability :", candidates[p[0]])
        test_ex[error_index] = candidates[p[0]]
        print("Corrected sentence:", test_ex)
    else:
        # Find candidate with highest unigram value
        probability_unigram = {}
        for i in range(len(candidates)):
            unigram_words= candidates[i]
            probability_unigram[i] = unigram(unigram_words)/len(train)
        q=Counter(probability_unigram).most_common()[0]
        print("Corrected word with high unigram probability :",candidates[q[0]])
        # yield best_candidate
        test_ex[error_index] = candidates[q[0]]
        print(test_ex)
               
# ** you should have implemented this as a function, like in the cell in Task 4 
# ** I'd take 2 off for that, except you did put the function in for Task 4, so only (-1)

Words mispelled :  ['calld']
Candidates with minimum edit distance : ['call', 'calls', 'called']
Corrected word with high bigram probability : called
Corrected sentence: ['so', 'they', 'called', 'it', 'the', 'murder', 'car']


## **Task 4**: 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your systems output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [332]:
# ** I can't figure this out. You should have done task 3 as a function 
# ** This should have been your answer for Task 3.. 
def acorrect(tokens):
    token_incorrect = []
    print(tokens)
    for token in tokens:
        j=1
        for i in all_words:
            if token == i:
                j=0
        if j==1:
            token_incorrect.append(token)
    print("Incorrect token is : ", token_incorrect)             
    #return token_incorrect


    #is_misspelled=acorrect(["so", "thy", "calld", "it", "the", "murder", "car"])
    is_misspelled=token_incorrect
    #test_ex = ["so", "thy", "calld", "it", "the", "murder", "car"]
    test_ex = tokens
    print("Words mispelled : ", is_misspelled)

    for i in range(len(is_misspelled)):
        # if is_misspelled(token):
        error_index=test_ex.index(is_misspelled[i])        
        print("Error Index : ",error_index, ":",is_misspelled[i])
        candidates = get_candidates(str(is_misspelled[i]))
        print("Candidates with minimum edit distance :",candidates)
        
        from operator import itemgetter
        if len(candidates) > 1:
            print("in IF condition Greater Candidate :")
            if error_index != 0:
                print("in IF condition - calculate Bigram :",is_misspelled[i])
                # Find candidate with highest bigram value
                probability_bigram = {}
                for i in range(len(candidates)):
                    print("Previous Error value :",test_ex[error_index-1])
                    bigram_words= test_ex[error_index-1] + " " + candidates[i]
                    print("Bi-gram words :",bigram_words)                    
                    #if unigram(test_ex[error_index-1]) > 0:
                    #print("in Uni gram Condition satisfied....")
                    unigramCount = unigram(test_ex[error_index-1])
                    if unigramCount == 0:
                        print("Unigram Count :",unigramCount)
                    else :                        
                        probability_bigram[i] = bigram(bigram_words)/unigram(test_ex[error_index-1])
                        print("prob for each bigram:",probability_bigram[i]) 
                        p=Counter(probability_bigram).most_common()[0]   
                        print("Corrected word with high bigram probability :", candidates[p[0]])
                        test_ex[error_index] = candidates[p[0]]
                        print("Corrected sentence:", test_ex)
                        
                print("Out of FOR LOOP")    
                             
            else:
                print("in ELSE condition - Unigram :",is_misspelled[i])
                # Find candidate with highest unigram value
                probability_unigram = {}
                for i in range(len(candidates)):
                    unigram_words= candidates[i]
                    probability_unigram[i] = unigram(unigram_words)/len(train)
                q=Counter(probability_unigram).most_common()[0]
                print("Corrected word with high unigram probability :",candidates[q[0]])
                # yield best_candidate
                test_ex[error_index] = candidates[q[0]]
                print(test_ex)
        else:
            print("in ELSE condition NO Candidates :")
            test_ex[error_index] = candidates[0]
            print(test_ex)
    return test_ex


In [333]:
is_misspelled_test = []
# ** why not just use provided tokenisations: [d.tokens_with_errors() for d in test] 
# ** why not just use provided tokenisations: [d.tokens_corrected() for d in test] (-1)
test_set = [d.text for d in test]
#is_misspelled_train=correct(train_set.split(" "))
Words_mispelled1=[]
#print("Words mispelled : ", is_misspelled_train)
for sentence in range(len(test_set)):
    s=test_set[sentence]
    line = s.rstrip('\r\n').replace(".","").strip() 
    if len(line) != 0:
        is_misspelled_test = line.split(" ")
        Words_mispelled1.append(acorrect(is_misspelled_test))

print(Words_mispelled1)

#for i in Words_mispelled1:
    #candidates = get_candidates(str(i))
#print("Candidates with minimum edit distance :",candidates)
    


['NIGEL', 'THRUSH', 'page', '48']
Incorrect token is :  ['NIGEL', 'THRUSH', '48']
Words mispelled :  ['NIGEL', 'THRUSH', '48']
Error Index :  0 : NIGEL
Candidates with minimum edit distance : ['ROGER', 'WIGGLE', 'DIED', 'JILL']
in IF condition Greater Candidate :
in ELSE condition - Unigram : NIGEL
Corrected word with high unigram probability : ROGER
['ROGER', 'THRUSH', 'page', '48']
Error Index :  1 : THRUSH
Candidates with minimum edit distance : ['HIS', 'THEN', 'JUST', 'THE', 'ROSE']
in IF condition Greater Candidate :
in IF condition - calculate Bigram : THRUSH
Previous Error value : ROGER
Bi-gram words : ROGER HIS
Unigram Count : 0
Previous Error value : ROGER
Bi-gram words : ROGER THEN
Unigram Count : 0
Previous Error value : ROGER
Bi-gram words : ROGER JUST
Unigram Count : 0
Previous Error value : ROGER
Bi-gram words : ROGER THE
Unigram Count : 0
Previous Error value : ROGER
Bi-gram words : ROGER ROSE
Unigram Count : 0
Out of FOR LOOP
Error Index :  3 : 48
Candidates with minimu

In [326]:
# ** this and the previous cell contain mostly duplicate code - better to build both arrays at once! (-1)

is_misspelled_test = []
test_set = [d.text for d in test]
#is_misspelled_train=correct(train_set.split(" "))
Words_mispelled2=[]
#print("Words mispelled : ", is_misspelled_train)
for sentence in range(len(test_set)):
    s=test_set[sentence]
    line = s.rstrip('\r\n').replace(".","").strip()
    #print(line)
    if len(line) != 0:
        Words_mispelled2.append(list(read_sentence(line).tokens_corrected()))

#print(Words_mispelled2)

   

In [334]:
Accuracy_count = 0
for i in range(len(Words_mispelled1)):
    #print("Mystatement:",i,":",Words_mispelled1[i])
    #print("Comparestatement:",i,":",Words_mispelled2[i])
    
    # ** you're calculating sentence accuracy, not word accuracy! (-5)
    # ** you need to loop over the words in each sentence also...
    if(Words_mispelled1[i]==Words_mispelled2[i]):
        Accuracy_count+=1
        #print(Accuracy_count)
Accuracy=( Accuracy_count/len(Words_mispelled1))*100
print( Accuracy)


24.175824175824175


Notice that your system currently corrects many words that do not need correction, the following code detects all the words that are in the test but not the training set and if they are errors or not:

In [9]:
# True spelling mistakes
true_positives = []
# Correctly spelled but unrecognized words
false_positives = []
false_negatives = []

for sentence in test:
    for word in sentence.tokens_with_errors():
        if word not in all_words:
            if word in sentence.errors():
                true_positives.append(word)
            else:
                false_positives.append(word)
        
        else:
            if word in sentence.errors():
                false_negatives.append(word)

                
print(len(true_positives))
print(len(false_positives))
print(len(false_negatives))

72
88
32


## **Task 5**: 
Using an SVM (see Lab 3) write a classifier that classifies a novel word as a misspelling or a new word based on the following features

* The minimum edit distance to the nearest known English word
* Whether it starts with a capital
* Whether it is all in capitals
* One other feature of your choice

In [118]:
def ed(token):
    ed = []
    for t in all_words:
        ed.append(edit_distance(token, t))
    min_ed = min(ed)
    return min_ed

def init_cap(token):
    
    if(token[0].isupper()):
        return 1
    else:
        return 0

def all_caps(token):
   
    if(all(word[0].isupper() for word in token)):
        return 1
    else:
        return 0 

# ** Your feature is the same as all_caps! (0/5)
def my_feature(token):
    # ** NOTE: `... for word in token` iterates over the CHARACTERS of token 
    # **  (ie: the variable `word` is a single character string)
    if (any(word[0].isupper() for word in token)):
        return 1
    else:
        return 0
    
X = [[ed(token), init_cap(token), all_caps(token), my_feature(token)] 
     for token in true_positives + false_positives]
y = [1] * len(true_positives) + [0] * len(false_positives)

from sklearn import svm

svc = svm.SVC()
svc_fit=svc.fit(X, y)
print(svc_fit)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


## **Task 6**: 
Using the `cross_validation` methods from Scikit Learn (covered in Lab 4) perform 10-fold cross validation and report your precision, recall and F-Measure.

In [122]:
from sklearn import cross_validation
scores = cross_validation.cross_val_score(svc_fit, X, y, cv=10)

# ** these are results on the training data, not from cross validation!! (-10)
precision=len(true_positives)/(len(true_positives)+len(false_positives))
print(precision)
recall=len(true_positives)/(len(true_positives)+len(false_negatives))
print(recall)
F_Measure=2*precision*recall/(precision+recall)
print(F_Measure)


0.45
0.6923076923076923
0.5454545454545455
